In [1]:
'''import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import implicit


product_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/products_data.pkl'
user_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/user_data.csv'
train_enriched_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/train_preprocessed.pkl"
test_enriched_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/test_preprocessed.pkl"


# Cargar los datasets
products = pd.read_pickle(product_path)
users = pd.read_csv(user_path)
train = pd.read_pickle(train_enriched_path)
test = pd.read_pickle(test_enriched_path)

# Convertir a numérico
train['pagetype'] = pd.to_numeric(train['pagetype'], errors='coerce')
# Reemplazar los NaN y asignar el resultado de vuelta a la columna
train['pagetype'] = train['pagetype'].fillna(-1)
# Cambiar el tipo de dato
train['pagetype'] = train['pagetype'].astype('int16')
train_user_ids = set(train['user_id'].unique())
train_user_ids.discard(-1)  # Remover -1 si representa usuarios no logueados
# Asumimos que si hay 'partnumber', hay interacción
test_sessions_with_interactions = set(test[test['partnumber'].notnull()]['session_id'].unique())



def classify_session(row):
    user_id = row['user_id']
    session_id = row['session_id']
    
    if user_id == -1:
        # Usuario no logueado
        if session_id in test_sessions_with_interactions:
            return 'Usuario recurrente no logueado'
        else:
            return 'Usuario nuevo no logueado'
    else:
        # Usuario logueado
        if user_id in train_user_ids:
            return 'Usuario recurrente logueado'
        else:
            return 'Usuario nuevo logueado'


test['user_class'] = test.apply(classify_session, axis=1)

# Modelo de Popularidad
# Calcular la popularidad de los productos en el conjunto de entrenamiento
product_popularity = train.groupby('partnumber')['add_to_cart'].sum().reset_index()
product_popularity.rename(columns={'add_to_cart': 'popularity'}, inplace=True)

# Ordenar los productos por popularidad descendente
product_popularity.sort_values(by='popularity', ascending=False, inplace=True)

# Obtener la lista de productos populares
popular_products = product_popularity['partnumber'].tolist()



# Asegurarnos de que los embeddings son arrays de NumPy
products['embedding'] = products['embedding'].apply(np.array)

# Crear un diccionario {partnumber: embedding}
embeddings_dict = dict(zip(products['partnumber'], products['embedding']))

def find_similar_products(partnumber, top_n=5):
    target_embedding = embeddings_dict.get(partnumber)
    if not isinstance(target_embedding, np.ndarray):
        print(f"Advertencia: El embedding del producto {partnumber} es inválido.")
        return []

    # Obtener todas las embeddings y los partnumbers correspondientes
    all_partnumbers = []
    all_embeddings = []
    for pnum, emb in embeddings_dict.items():
        if isinstance(emb, np.ndarray) and emb.shape == target_embedding.shape:
            all_partnumbers.append(pnum)
            all_embeddings.append(emb)

    # Convertir a arrays de NumPy
    all_embeddings = np.stack(all_embeddings)

    # Calcular la similitud de coseno
    similarities = cosine_similarity([target_embedding], all_embeddings)[0]

    # Obtener los índices de los productos más similares (excluyendo el propio producto)
    similar_indices = similarities.argsort()[::-1]
    similar_partnumbers = []
    for idx in similar_indices:
        if all_partnumbers[idx] != partnumber:
            similar_partnumbers.append(all_partnumbers[idx])
        if len(similar_partnumbers) == top_n:
            break

    return similar_partnumbers

# Seleccionar un producto para probar, por ejemplo, el más popular
test_partnumber = popular_products[0]  # Primer producto de la lista de populares

# Obtener productos similares
similar_products = find_similar_products(test_partnumber, top_n=5)


# Filtrar interacciones de usuarios logueados y crear una copia
train_logged_in = train[train['user_id'] != -1].copy()


# Obtener user_ids únicos
unique_user_ids = train_logged_in['user_id'].unique()

# Crear diccionarios de mapeo
user_id_to_index = {user_id: idx for idx, user_id in enumerate(unique_user_ids)}
index_to_user_id = {idx: user_id for user_id, idx in user_id_to_index.items()}

# Mapear 'user_id' a 'user_idx' en train_logged_in
train_logged_in['user_idx'] = train_logged_in['user_id'].map(user_id_to_index)

# Obtener partnumbers únicos
unique_partnumbers = train_logged_in['partnumber'].unique()

# Crear diccionarios de mapeo
partnumber_to_index = {partnumber: idx for idx, partnumber in enumerate(unique_partnumbers)}
index_to_partnumber = {idx: partnumber for partnumber, idx in partnumber_to_index.items()}

# Mapear 'partnumber' a 'item_idx' en train_logged_in
train_logged_in['item_idx'] = train_logged_in['partnumber'].map(partnumber_to_index)

# Construir la matriz de interacciones (usuarios x items)
interaction_matrix = csr_matrix(
    (train_logged_in['add_to_cart'], (train_logged_in['user_idx'], train_logged_in['item_idx'])),
    shape=(len(unique_user_ids), len(unique_partnumbers))
)

# Configurar el modelo (no transponemos la matriz)
model = implicit.als.AlternatingLeastSquares(factors=50, iterations=10, regularization=0.1)

# Entrenar el modelo
model.fit(interaction_matrix)

user_idx = 0  # Puedes elegir cualquier índice válido
user_id = index_to_user_id[user_idx]

# Obtener las interacciones del usuario
user_items = interaction_matrix[user_idx]

# Obtener recomendaciones
recommended = model.recommend(user_idx, user_items, N=5)

if isinstance(recommended, list):
    # recommended es una lista de tuplas (item_idx, score)
    recommended_indices = [item_idx for item_idx, score in recommended]
elif isinstance(recommended, tuple) and len(recommended) == 2:
    # recommended es una tupla de (indices, scores)
    recommended_indices = recommended[0].tolist()
elif isinstance(recommended, np.ndarray):
    # recommended es un array de índices
    recommended_indices = recommended.tolist()
else:
    print("Formato de 'recommended' no reconocido.")
    recommended_indices = []
    
    
# Verificar si los índices existen en 'index_to_partnumber'
missing_indices = [idx for idx in recommended_indices if idx not in index_to_partnumber]

if missing_indices:
    print(f"Los siguientes índices no se encuentran en 'index_to_partnumber': {missing_indices}")
else:
    print("Todos los índices recomendados están en 'index_to_partnumber'")
    
    # Mapear los índices a 'partnumber'
recommended_partnumbers = [index_to_partnumber[idx] for idx in recommended_indices]

group = test[test['user_class'] == 'Usuario recurrente logueado']
total_rows = group.shape[0]
num_sessions = group['session_id'].nunique()

# Filtrar las sesiones de 'Usuario recurrente logueado'
group = test[test['user_class'] == 'Usuario recurrente logueado']

# Obtener las sesiones únicas y los user_id asociados
session_user_mapping = group[['session_id', 'user_id']].drop_duplicates()

def recommend_by_collaborative(user_id, top_n=5):
    # Obtener el índice del usuario
    user_idx = user_id_to_index.get(user_id)
    
    if user_idx is None:
        # Si el usuario no está en el modelo, retornamos una lista vacía o usamos popularidad
        print(f"Usuario {user_id} no encontrado en el modelo colaborativo.")
        return []
    
    # Obtener las interacciones del usuario
    user_items = interaction_matrix[user_idx]
    
    # Obtener recomendaciones del modelo
    recommended = model.recommend(user_idx, user_items, N=top_n)
    
    # Ajustar según el tipo de 'recommended'
    if isinstance(recommended, list):
        # recommended es una lista de tuplas (item_idx, score)
        recommended_indices = [item_idx for item_idx, score in recommended]
    elif isinstance(recommended, tuple) and len(recommended) == 2:
        # recommended es una tupla de (indices, scores)
        recommended_indices = recommended[0].tolist()
    elif isinstance(recommended, np.ndarray):
        # recommended es un array de índices
        recommended_indices = recommended.tolist()
    else:
        print("Formato de 'recommended' no reconocido.")
        recommended_indices = []
    
    # Verificar si los índices existen en 'index_to_partnumber'
    recommended_indices = [idx for idx in recommended_indices if idx in index_to_partnumber]
    
    # Mapear los índices a 'partnumber'
    recommended_partnumbers = [index_to_partnumber[idx] for idx in recommended_indices]
    
    return recommended_partnumbers

# Seleccionar un user_id de prueba (asegúrate de que el user_id existe en user_id_to_index)
user_id = unique_user_ids[0]
recommended_partnumbers = recommend_by_collaborative(user_id, top_n=5)

# Mantener el diccionario 'user_recommendations' si ya lo tienes definido
# De lo contrario, inicialízalo
user_recommendations = {}

# Iterar sobre cada sesión y user_id asociados
for _, row in session_user_mapping.iterrows():
    session_id = row['session_id']
    user_id = row['user_id']
    
    # Obtener recomendaciones colaborativas
    recs = recommend_by_collaborative(user_id, top_n=10)  # Usamos top_n=10 para tener margen
    
    # Obtener productos ya interactuados por el usuario en el conjunto de entrenamiento
    user_interacted_items = train_logged_in[train_logged_in['user_id'] == user_id]['partnumber'].unique()
    
    # Eliminar productos ya vistos
    recs = [p for p in recs if p not in user_interacted_items]
    
    # Si después de filtrar no tenemos suficientes recomendaciones, complementamos con popularidad
    if len(recs) < 5:
        more_recs = [p for p in popular_products if p not in recs and p not in user_interacted_items]
        recs.extend(more_recs[:5 - len(recs)])
    
    # Aseguramos 5 recomendaciones únicas
    recs = list(dict.fromkeys(recs))[:5]
    
    # Almacenar las recomendaciones
    user_recommendations[session_id] = recs
    
    
# Mostrar las recomendaciones para las primeras 5 sesiones
session_ids = list(user_recommendations.keys())
for session_id in session_ids[:5]:
    recs = user_recommendations[session_id]
    print(f"Sesión {session_id} - Recomendaciones: {recs}")
    

# Obtener el número total de sesiones en el conjunto de prueba
total_sessions_in_test = test['session_id'].nunique()

# Obtener el número total de sesiones para las que hemos generado recomendaciones
total_sessions_with_recommendations = len(user_recommendations)

print(f"Total de sesiones en el conjunto de prueba: {total_sessions_in_test}")
print(f"Total de sesiones con recomendaciones: {total_sessions_with_recommendations}")

if total_sessions_in_test == total_sessions_with_recommendations:
    print("Todas las sesiones tienen recomendaciones.")
else:
    print(f"Faltan recomendaciones para {total_sessions_in_test - total_sessions_with_recommendations} sesiones.")
    

# Filtrar las sesiones de 'Usuario recurrente no logueado'
group = test[test['user_class'] == 'Usuario recurrente no logueado']

# Obtener las sesiones únicas
unique_sessions = group['session_id'].unique()

def recommend_by_content(partnumbers_interacted, top_n=5):
    recommendations = []
    for partnumber in partnumbers_interacted:
        # Obtener productos similares
        similar_products = find_similar_products(partnumber, top_n=top_n)
        # Añadir los productos similares a la lista de recomendaciones
        recommendations.extend(similar_products)
    
    # Eliminar productos ya vistos y duplicados
    recommendations = [p for p in recommendations if p not in partnumbers_interacted]
    recommendations = list(dict.fromkeys(recommendations))
    
    return recommendations[:top_n]

# Seleccionar un 'partnumber' de ejemplo con el que haya interactuado un usuario
test_partnumbers_interacted = [40779]  # Este es el 'partnumber' más popular según nuestro dato previo

# Generar recomendaciones basadas en contenido
recs_content = recommend_by_content(test_partnumbers_interacted, top_n=5)


def recommend_by_popularity(top_n=5):
    return popular_products[:top_n]

# Continuamos usando el diccionario 'user_recommendations' existente

for session_id in unique_sessions:
    # Obtener los 'partnumber' con los que interactuó en la sesión
    session_data = group[group['session_id'] == session_id]
    partnumbers_interacted = session_data['partnumber'].unique()
    
    if len(partnumbers_interacted) > 0:
        # Generar recomendaciones basadas en contenido
        recs = recommend_by_content(partnumbers_interacted, top_n=10)
    else:
        # Si no hay interacciones, usar popularidad
        recs = recommend_by_popularity(top_n=5)
    
    # Aseguramos 5 recomendaciones únicas
    recs = list(dict.fromkeys(recs))[:5]
    
    # Almacenar las recomendaciones en el diccionario existente
    user_recommendations[session_id] = recs
    

# Filtrar las sesiones de 'Usuario nuevo logueado'
group_new_logged_in = test[test['user_class'] == 'Usuario nuevo logueado']

# Obtener las sesiones únicas y los 'user_id' asociados
session_user_mapping_new_logged_in = group_new_logged_in[['session_id', 'user_id']].drop_duplicates()

# Continuamos utilizando el diccionario 'user_recommendations'

for _, row in session_user_mapping_new_logged_in.iterrows():
    session_id = row['session_id']
    user_id = row['user_id']
    
    # Generar recomendaciones basadas en popularidad
    recs = recommend_by_popularity(top_n=5)
    
    # Almacenar las recomendaciones
    user_recommendations[session_id] = recs
    

# Filtrar las sesiones de 'Usuario nuevo no logueado'
group_new_not_logged_in = test[test['user_class'] == 'Usuario nuevo no logueado']

if not group_new_not_logged_in.empty:
    # Obtener las sesiones únicas
    unique_sessions_new_not_logged_in = group_new_not_logged_in['session_id'].unique()
    print(f"Número de sesiones únicas de 'Usuario nuevo no logueado': {len(unique_sessions_new_not_logged_in)}")
    
    for session_id in unique_sessions_new_not_logged_in:
        # Generar recomendaciones basadas en popularidad
        recs = recommend_by_popularity(top_n=5)
        
        # Almacenar las recomendaciones
        user_recommendations[session_id] = recs

    # Verificar el total de sesiones con recomendaciones
    total_sessions_with_recommendations = len(user_recommendations)
    print(f"Total de sesiones con recomendaciones después de 'Usuario nuevo no logueado': {total_sessions_with_recommendations}")
else:
    print("No hay sesiones clasificadas como 'Usuario nuevo no logueado'.")
    

import json
# Convertir las claves de 'user_recommendations' a cadenas
user_recommendations_str_keys = {str(session_id): recs for session_id, recs in user_recommendations.items()}

# Convertir los elementos de las listas de recomendaciones a enteros nativos de Python
for session_id, recs in user_recommendations_str_keys.items():
    user_recommendations_str_keys[session_id] = [int(p) for p in recs]

# Crear el dictado de salida
output = {'target': user_recommendations_str_keys}

# Guardar el archivo JSON
with open('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_3_pipeline_v2.json', 'w') as f:
    json.dump(output, f)

print("Archivo 'predictions_3_pipeline_v2.json' generado con éxito.")

with open('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions_save/predictions_3_pipeline_v2.json', 'r') as f:
    data = json.load(f)

# Verificar el número de sesiones
num_sessions_in_json = len(data['target'])
print(f"Número de sesiones en el JSON: {num_sessions_in_json}")

# Mostrar algunas recomendaciones
print("Primeras 5 sesiones en el JSON:")
for session_id in list(data['target'].keys())[:5]:
    recs = data['target'][session_id]
    print(f"Sesión {session_id}: {recs}")
    
'''

'import pandas as pd\nimport numpy as np\nfrom sklearn.metrics.pairwise import cosine_similarity\nfrom scipy.sparse import csr_matrix\nimport implicit\n\n\nproduct_path = \'/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/products_data.pkl\'\nuser_path = \'/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/user_data.csv\'\ntrain_enriched_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/train_preprocessed.pkl"\ntest_enriched_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/test_preprocessed.pkl"\n\n\n# Cargar los datasets\nproducts = pd.read_pickle(product_path)\nusers = pd.read_csv(user_path)\ntrain = pd.read_pickle(train_enriched_path)\ntest = pd.read_pickle(test_enriched_path)\n\n# Convertir a numérico\ntrain[\'pagetype\'] = pd.to_numer

---

# **Refractorización del pipeline**

## Bloque de Preparación

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import implicit
import json
import os

# Definir rutas de los archivos
product_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/products_data.pkl'
user_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/user_data.csv'
train_enriched_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/train_preprocessed.pkl"
test_enriched_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/hybrid_model/test_preprocessed.pkl"

# Cargar los datasets
products = pd.read_pickle(product_path)
users = pd.read_csv(user_path)
train = pd.read_pickle(train_enriched_path)
test = pd.read_pickle(test_enriched_path)

# Preprocesamiento de datos
def preprocess_data(train, test):
    # Convertir 'pagetype' a numérico en 'train'
    train['pagetype'] = pd.to_numeric(train['pagetype'], errors='coerce')
    train['pagetype'] = train['pagetype'].fillna(-1)
    train['pagetype'] = train['pagetype'].astype('int16')
    
    # Obtener los user_id únicos en 'train'
    train_user_ids = set(train['user_id'].unique())
    train_user_ids.discard(-1)  # Remover -1 si representa usuarios no logueados

    # Identificar sesiones con interacciones en 'test'
    test_sessions_with_interactions = set(test[test['partnumber'].notnull()]['session_id'].unique())
    
    return train_user_ids, test_sessions_with_interactions

# Clasificar sesiones
def classify_sessions(test, train_user_ids, test_sessions_with_interactions):
    def classify_session(row):
        user_id = row['user_id']
        session_id = row['session_id']
        
        if user_id == -1:
            # Usuario no logueado
            if session_id in test_sessions_with_interactions:
                return 'Usuario recurrente no logueado'
            else:
                return 'Usuario nuevo no logueado'
        else:
            # Usuario logueado
            if user_id in train_user_ids:
                return 'Usuario recurrente logueado'
            else:
                return 'Usuario nuevo logueado'
    test['user_class'] = test.apply(classify_session, axis=1)
    return test

# Ejecutar preprocesamiento y clasificación
train_user_ids, test_sessions_with_interactions = preprocess_data(train, test)
test = classify_sessions(test, train_user_ids, test_sessions_with_interactions)

/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/.env/lib/python3.10/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


## Funciones de cada Modelo

### Modelo de Popularidad

In [3]:
def train_popularity_model(train):
    # Calcular la popularidad de los productos en el conjunto de entrenamiento
    product_popularity = train.groupby('partnumber')['add_to_cart'].sum().reset_index()
    product_popularity.rename(columns={'add_to_cart': 'popularity'}, inplace=True)
    product_popularity.sort_values(by='popularity', ascending=False, inplace=True)
    popular_products = product_popularity['partnumber'].tolist()
    return popular_products

# Entrenar el modelo de popularidad
popular_products = train_popularity_model(train)

def recommend_by_popularity(popular_products, top_n=5):
    return popular_products[:top_n]

### Modelo Basado en Contenido

In [4]:
def prepare_content_model(products):
    # Asegurarnos de que los embeddings son arrays de NumPy
    products['embedding'] = products['embedding'].apply(np.array)
    # Crear un diccionario {partnumber: embedding}
    embeddings_dict = dict(zip(products['partnumber'], products['embedding']))
    return embeddings_dict

def find_similar_products(partnumber, embeddings_dict, top_n=5):
    target_embedding = embeddings_dict.get(partnumber)
    if not isinstance(target_embedding, np.ndarray):
        print(f"Advertencia: El embedding del producto {partnumber} es inválido.")
        return []

    # Obtener todas las embeddings y los partnumbers correspondientes
    all_partnumbers = []
    all_embeddings = []
    for pnum, emb in embeddings_dict.items():
        if isinstance(emb, np.ndarray) and emb.shape == target_embedding.shape:
            all_partnumbers.append(pnum)
            all_embeddings.append(emb)

    # Convertir a arrays de NumPy
    all_embeddings = np.stack(all_embeddings)

    # Calcular la similitud de coseno
    similarities = cosine_similarity([target_embedding], all_embeddings)[0]

    # Obtener los índices de los productos más similares (excluyendo el propio producto)
    similar_indices = similarities.argsort()[::-1]
    similar_partnumbers = []
    for idx in similar_indices:
        if all_partnumbers[idx] != partnumber:
            similar_partnumbers.append(all_partnumbers[idx])
        if len(similar_partnumbers) == top_n:
            break

    return similar_partnumbers

def recommend_by_content(partnumbers_interacted, embeddings_dict, top_n=5):
    recommendations = []
    for partnumber in partnumbers_interacted:
        # Obtener productos similares
        similar_products = find_similar_products(partnumber, embeddings_dict, top_n=top_n)
        # Añadir los productos similares a la lista de recomendaciones
        recommendations.extend(similar_products)

    # Eliminar productos ya vistos y duplicados
    recommendations = [p for p in recommendations if p not in partnumbers_interacted]
    recommendations = list(dict.fromkeys(recommendations))

    return recommendations[:top_n]

# Preparar el modelo basado en contenido
embeddings_dict = prepare_content_model(products)

### Modelo Colaborativo

In [5]:
def prepare_collaborative_model(train_logged_in):
    # Obtener user_ids únicos
    unique_user_ids = train_logged_in['user_id'].unique()
    # Crear diccionarios de mapeo
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(unique_user_ids)}
    index_to_user_id = {idx: user_id for user_id, idx in user_id_to_index.items()}
    # Mapear 'user_id' a 'user_idx' en train_logged_in
    train_logged_in['user_idx'] = train_logged_in['user_id'].map(user_id_to_index)
    # Obtener partnumbers únicos
    unique_partnumbers = train_logged_in['partnumber'].unique()
    # Crear diccionarios de mapeo
    partnumber_to_index = {partnumber: idx for idx, partnumber in enumerate(unique_partnumbers)}
    index_to_partnumber = {idx: partnumber for partnumber, idx in partnumber_to_index.items()}
    # Mapear 'partnumber' a 'item_idx' en train_logged_in
    train_logged_in['item_idx'] = train_logged_in['partnumber'].map(partnumber_to_index)
    # Construir la matriz de interacciones (usuarios x items)
    interaction_matrix = csr_matrix(
        (train_logged_in['add_to_cart'], (train_logged_in['user_idx'], train_logged_in['item_idx'])),
        shape=(len(unique_user_ids), len(unique_partnumbers))
    )
    # Configurar el modelo (no transponemos la matriz)
    model = implicit.als.AlternatingLeastSquares(factors=50, iterations=10, regularization=0.1)
    # Entrenar el modelo
    model.fit(interaction_matrix)
    return model, interaction_matrix, user_id_to_index, index_to_user_id, index_to_partnumber

def recommend_by_collaborative(user_id, model, interaction_matrix, user_id_to_index, index_to_partnumber, top_n=5):
    # Obtener el índice del usuario
    user_idx = user_id_to_index.get(user_id)
    if user_idx is None:
        # Si el usuario no está en el modelo, retornamos una lista vacía o usamos popularidad
        print(f"Usuario {user_id} no encontrado en el modelo colaborativo.")
        return []

    # Obtener las interacciones del usuario
    user_items = interaction_matrix[user_idx]

    # Obtener recomendaciones del modelo
    recommended = model.recommend(user_idx, user_items, N=top_n)

    # Ajustar según el tipo de 'recommended'
    if isinstance(recommended, list):
        # recommended es una lista de tuplas (item_idx, score)
        recommended_indices = [item_idx for item_idx, score in recommended]
    elif isinstance(recommended, tuple) and len(recommended) == 2:
        # recommended es una tupla de (indices, scores)
        recommended_indices = recommended[0].tolist()
    elif isinstance(recommended, np.ndarray):
        # recommended es un array de índices
        recommended_indices = recommended.tolist()
    else:
        print("Formato de 'recommended' no reconocido.")
        recommended_indices = []

    # Verificar si los índices existen en 'index_to_partnumber'
    recommended_indices = [idx for idx in recommended_indices if idx in index_to_partnumber]

    # Mapear los índices a 'partnumber'
    recommended_partnumbers = [index_to_partnumber[idx] for idx in recommended_indices]

    return recommended_partnumbers

# Preparar el modelo colaborativo
train_logged_in = train[train['user_id'] != -1].copy()
model_collaborative, interaction_matrix, user_id_to_index, index_to_user_id, index_to_partnumber = prepare_collaborative_model(train_logged_in)

/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/.env/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 10 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 10/10 [00:08<00:00,  1.12it/s]


## Generación de recomendaciones

In [6]:
# Función para generar recomendaciones por sesión
def generate_recommendations_for_session(session_id, user_id, session_data, user_class):
    if user_class == 'Usuario recurrente logueado':
        # Usar modelo colaborativo
        recs = recommend_by_collaborative(
            user_id, model_collaborative, interaction_matrix, user_id_to_index, index_to_partnumber, top_n=10
        )
        # Obtener productos ya interactuados por el usuario en el conjunto de entrenamiento
        user_interacted_items = train_logged_in[train_logged_in['user_id'] == user_id]['partnumber'].unique()
        # Eliminar productos ya vistos
        recs = [p for p in recs if p not in user_interacted_items]
        # Si después de filtrar no tenemos suficientes recomendaciones, complementamos con popularidad
        if len(recs) < 5:
            more_recs = [p for p in popular_products if p not in recs and p not in user_interacted_items]
            recs.extend(more_recs[:5 - len(recs)])
    elif user_class == 'Usuario recurrente no logueado':
        # Usar modelo basado en contenido
        partnumbers_interacted = session_data['partnumber'].unique()
        if len(partnumbers_interacted) > 0:
            recs = recommend_by_content(partnumbers_interacted, embeddings_dict, top_n=10)
        else:
            recs = recommend_by_popularity(popular_products, top_n=5)
    else:  # 'Usuario nuevo logueado' o 'Usuario nuevo no logueado'
        # Usar popularidad
        recs = recommend_by_popularity(popular_products, top_n=5)
    # Asegurar 5 recomendaciones únicas
    recs = list(dict.fromkeys(recs))[:5]
    return recs

# Generar recomendaciones para todas las sesiones
def generate_all_recommendations(test):
    user_recommendations = {}
    # Iterar sobre todas las sesiones únicas
    sessions = test[['session_id', 'user_id', 'user_class']].drop_duplicates()

    for _, row in sessions.iterrows():
        session_id = row['session_id']
        user_id = row['user_id']
        user_class = row['user_class']
        session_data = test[test['session_id'] == session_id]
        recs = generate_recommendations_for_session(session_id, user_id, session_data, user_class)
        user_recommendations[session_id] = recs
    return user_recommendations

# Generar todas las recomendaciones
user_recommendations = generate_all_recommendations(test)

# Verificación de que todas las sesiones tienen recomendaciones
total_sessions_in_test = test['session_id'].nunique()
total_sessions_with_recommendations = len(user_recommendations)

print(f"Total de sesiones en el conjunto de prueba: {total_sessions_in_test}")
print(f"Total de sesiones con recomendaciones: {total_sessions_with_recommendations}")

if total_sessions_in_test == total_sessions_with_recommendations:
    print("Todas las sesiones tienen recomendaciones.")
else:
    print(f"Faltan recomendaciones para {total_sessions_in_test - total_sessions_with_recommendations} sesiones.")

KeyboardInterrupt: 

## Generación del Json

In [ ]:
# Preparar los datos para el JSON
def prepare_output_for_json(user_recommendations):
    # Convertir las claves de 'user_recommendations' a cadenas
    user_recommendations_str_keys = {str(session_id): recs for session_id, recs in user_recommendations.items()}
    # Convertir los elementos de las listas de recomendaciones a enteros nativos de Python
    for session_id, recs in user_recommendations_str_keys.items():
        user_recommendations_str_keys[session_id] = [int(p) for p in recs]
    return user_recommendations_str_keys

# Guardar el archivo JSON
def save_recommendations_to_json(user_recommendations_str_keys, output_path):
    output = {'target': user_recommendations_str_keys}
    with open(output_path, 'w') as f:
        json.dump(output, f)
    print(f"Archivo '{output_path}' generado con éxito.")

# Ruta de salida para el JSON
output_json_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_3_pipeline_v2.json'

# Preparar y guardar el JSON
user_recommendations_str_keys = prepare_output_for_json(user_recommendations)
save_recommendations_to_json(user_recommendations_str_keys, output_json_path)

# Verificar el contenido del JSON
def verify_json_output(output_path):
    with open(output_path, 'r') as f:
        data = json.load(f)
    # Verificar el número de sesiones
    num_sessions_in_json = len(data['target'])
    print(f"Número de sesiones en el JSON: {num_sessions_in_json}")
    # Mostrar algunas recomendaciones
    print("Primeras 5 sesiones en el JSON:")
    for session_id in list(data['target'].keys())[:5]:
        recs = data['target'][session_id]
        print(f"Sesión {session_id}: {recs}")

# Verificar el JSON generado
verify_json_output(output_json_path)